In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
import torch.nn as nn 
import torch.nn.functional as F  
import safetensors
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
from tqdm import tqdm
from itertools import permutations
import warnings
warnings.filterwarnings('ignore')

c:\Anaconda3\envs\daycon\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Anaconda3\envs\daycon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_label_mappings():
    """24가지 순열에 대한 라벨 매핑 생성"""
    all_permutations = list(permutations([0, 1, 2, 3]))
    perm_to_label = {perm: idx for idx, perm in enumerate(all_permutations)}
    label_to_perm = {idx: perm for idx, perm in enumerate(all_permutations)}
    
    print(f"총 {len(all_permutations)}개의 순열 클래스 생성")
    print("예시 매핑:")
    for i in range(5):
        print(f"  라벨 {i}: {all_permutations[i]}")
    
    return perm_to_label, label_to_perm

# 순열 매핑 생성
perm_to_label, label_to_perm = create_label_mappings()

총 24개의 순열 클래스 생성
예시 매핑:
  라벨 0: (0, 1, 2, 3)
  라벨 1: (0, 1, 3, 2)
  라벨 2: (0, 2, 1, 3)
  라벨 3: (0, 2, 3, 1)
  라벨 4: (0, 3, 1, 2)


In [3]:
def prepare_roberta_data(train_df, perm_to_label):
    """RoBERTa용 데이터 준비"""
    processed_data = []
    
    for _, row in train_df.iterrows():
        sentences = [row[f"sentence_{i}"] for i in range(4)]
        answer_tuple = tuple([row[f"answer_{i}"] for i in range(4)])
        text = " [SEP] ".join(sentences)
        label = perm_to_label[answer_tuple]
        
        processed_data.append({
            "text": text,
            "label": label,
            "original_sentences": sentences,
            "answer": answer_tuple
        })
    
    return processed_data

def augment_roberta_data_advanced(train_df, perm_to_label, multiplier=4):
    """고급 데이터 증강"""
    augmented_data = []
    
    # 원본 데이터
    original_data = prepare_roberta_data(train_df, perm_to_label)
    augmented_data.extend(original_data)
    
    # 다양한 증강 방법
    for aug_round in range(multiplier - 1):
        for _, row in train_df.iterrows():
            sentences = [row[f"sentence_{i}"] for i in range(4)]
            original_answer = [row[f"answer_{i}"] for i in range(4)]
            
            if aug_round == 0:
                # 랜덤 셔플
                indices = list(range(4))
                np.random.shuffle(indices)
            elif aug_round == 1:
                # 순환 이동
                shift = np.random.randint(1, 4)
                indices = [(i + shift) % 4 for i in range(4)]
            else:
                # 부분 교환
                indices = list(range(4))
                i, j = np.random.choice(4, 2, replace=False)
                indices[i], indices[j] = indices[j], indices[i]
            
            shuffled_sentences = [sentences[i] for i in indices]
            new_answer = tuple([indices.index(original_answer[i]) for i in range(4)])
            
            text = " [SEP] ".join(shuffled_sentences)
            label = perm_to_label[new_answer]
            
            augmented_data.append({
                "text": text,
                "label": label,
                "original_sentences": shuffled_sentences,
                "answer": new_answer
            })
    
    print(f"고급 데이터 증강 완료: {len(original_data)} → {len(augmented_data)}")
    return augmented_data

In [4]:
print("데이터 로드 중...")
train_df = pd.read_csv('./train.csv')
print(f"원본 학습 데이터: {len(train_df)}개")

print("\n고급 데이터 증강 중...")
augmented_data = augment_roberta_data_advanced(train_df, perm_to_label, multiplier=4)

print("\n학습/검증 데이터 분할...")
train_data, valid_data = train_test_split(
    augmented_data, 
    test_size=0.2, 
    random_state=42,
    stratify=[item["label"] for item in augmented_data]
)

print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(valid_data)}개")

데이터 로드 중...
원본 학습 데이터: 7351개

고급 데이터 증강 중...
고급 데이터 증강 완료: 7351 → 29404

학습/검증 데이터 분할...
학습 데이터: 23523개
검증 데이터: 5881개


In [5]:
import torch.nn.functional as F

class ListwiseRankingLoss(nn.Module):
    """논문에서 검증된 가장 효과적인 ListMLE 손실"""
    
    def __init__(self, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        
    def forward(self, logits, labels):
        """
        logits: [batch_size, num_classes] - 모델 출력 (24개 순열에 대한 점수)
        labels: [batch_size] - 정답 순열 인덱스
        """
        batch_size = logits.size(0)
        num_classes = logits.size(1)  # 24
        
        # 각 배치에 대해 ListMLE 손실 계산
        total_loss = 0
        
        for batch_idx in range(batch_size):
            batch_logits = logits[batch_idx] / self.temperature  # [24]
            target_label = labels[batch_idx].item()
            
            # 정답 순열을 실제 순서로 변환
            target_permutation = self.label_to_permutation(target_label)
            
            # ListMLE: 순차적으로 각 위치에서 올바른 문장을 선택할 확률
            remaining_positions = list(range(4))
            listwise_loss = 0
            
            for pos in range(4):
                correct_sentence = target_permutation[pos]
                
                if correct_sentence in remaining_positions:
                    # 남은 문장들 중에서 이 위치에 올 확률
                    # 여기서는 단순화: 전체 순열 확률로 근사
                    position_probs = F.softmax(batch_logits, dim=0)
                    listwise_loss += -torch.log(position_probs[target_label] + 1e-8)
                    break  # 단순화된 버전
            
            total_loss += listwise_loss
        
        return total_loss / batch_size
    
    def label_to_permutation(self, label):
        """라벨을 순열로 변환하는 함수 (전역 변수 사용)"""
        # 전역 label_to_perm 딕셔너리 사용
        return label_to_perm[label]

In [6]:
class AdvancedStackingEnsemble:
    """Listwise Ranking Loss가 적용된 최고 성능 Stacking 앙상블"""
    
    def __init__(self, n_folds=5, random_state=42):
        self.n_folds = n_folds
        self.random_state = random_state
        self.base_models = []
        self.meta_model = None
        # 🚀 연구 기반 최적화된 설정 + ELECTRA base 업그레이드
        self.base_model_configs = [
            {
                'name': 'electra_small_listwise',   # 🚀 가벼운 ELECTRA Small
                'model_name': 'monologg/koelectra-small-v3-discriminator',
                'learning_rate': 3e-5,        # small 모델은 높은 LR
                'epochs': 45,                 # 🔥 충분한 학습 (20→45로 대폭 증가!)
                'batch_size': 64,             # small이므로 배치 증가 가능
                'warmup_steps': 300,
                'weight_decay': 0.01,
                'max_grad_norm': 1.0,
                'early_stopping_patience': 8  # 에포크가 많아졌으니 patience도 증가
            },
            {
                'name': 'bert_base_listwise', 
                'model_name': 'klue/bert-base',
                'learning_rate': 2e-5,        # 안정적
                'epochs': 40,                 # 🔥 충분한 학습
                'batch_size': 32,             # FP16 제거로 배치 사이즈 조정
                'warmup_steps': 400,
                'weight_decay': 0.01,
                'max_grad_norm': 1.0,
                'early_stopping_patience': 8   # 충분히 기다림
            },
            {
                'name': 'roberta_small_listwise',
                'model_name': 'klue/roberta-small',
                'learning_rate': 2e-5,        # 안정화
                'epochs': 35,                 # 🔥 충분한 학습
                'batch_size': 48,             # FP16 제거로 조정
                'warmup_steps': 300,
                'weight_decay': 0.01,
                'max_grad_norm': 1.0,
                'early_stopping_patience': 6
            }
        ]
    
    def train_base_model(self, config, train_dataset, valid_dataset, tokenizer, device):
        """Listwise Loss가 적용된 베이스 모델 학습"""
        print(f"\n🔥 {config['name']} 모델 학습 시작 (Listwise Loss 적용)...")
        
        # 모델 생성 (FP16 제거)
        model = AutoModelForSequenceClassification.from_pretrained(
            config['model_name'],
            num_labels=24,
            cache_dir='C:/huggingface_cache'
        )
        model.to(device)
        
        # 🎯 최적화된 학습 설정
        training_args = TrainingArguments(
            output_dir=f"./results_{config['name']}_listwise",
            
            # 핵심 하이퍼파라미터
            learning_rate=config['learning_rate'],
            per_device_train_batch_size=config['batch_size'],
            per_device_eval_batch_size=64,
            num_train_epochs=config['epochs'],
            
            # 최적화 설정
            warmup_steps=config['warmup_steps'],
            weight_decay=config['weight_decay'],
            max_grad_norm=config['max_grad_norm'],
            
            # 성능 향상 설정
            gradient_accumulation_steps=2,
            dataloader_pin_memory=True,
            dataloader_num_workers=4,
            group_by_length=True,
            
            # 평가 및 저장 (더 자주)
            eval_strategy="steps",
            eval_steps=100,
            save_strategy="steps",
            save_steps=100,
            save_total_limit=3,
            
            # 모델 선택
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,
            
            # 시스템 최적화 (FP16 제거)
            dataloader_drop_last=True,
            remove_unused_columns=False,
            
            # 로깅
            logging_steps=50,
            report_to=None,
            seed=42,
            data_seed=42,
        )
        
        # 🚀 조기 종료 설정
        from transformers import EarlyStoppingCallback
        callbacks = [EarlyStoppingCallback(
            early_stopping_patience=config['early_stopping_patience']
        )]
        
        # 🎯 Listwise Loss를 사용하는 커스텀 트레이너
        class ListwiseTrainer(Trainer):
            def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)
                self.listwise_loss = ListwiseRankingLoss(temperature=1.0)
            
            def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
                # 모든 추가 키워드 인자들을 받도록 **kwargs 추가
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                
                # Listwise Ranking Loss 적용
                loss = self.listwise_loss(logits, labels)
                
                return (loss, outputs) if return_outputs else loss
        
        # 트레이너 생성 (Listwise Loss 적용)
        trainer = ListwiseTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=valid_dataset,
            tokenizer=tokenizer,
            data_collator=DataCollatorWithPadding(
                tokenizer=tokenizer,
                padding=True,
                max_length=512,
                pad_to_multiple_of=8
            ),
            compute_metrics=lambda eval_pred: {
                "accuracy": accuracy_score(
                    eval_pred.label_ids, 
                    np.argmax(eval_pred.predictions, axis=1)
                )
            },
            callbacks=callbacks
        )
        
        # 📊 학습 파라미터 출력
        print(f"🎓 {config['name']} Listwise 설정:")
        print(f"   Learning Rate: {config['learning_rate']}")
        print(f"   Batch Size: {config['batch_size']}")
        print(f"   Epochs: {config['epochs']}")
        print(f"   Loss Function: ListMLE (Listwise Ranking)")
        print(f"   Early Stopping: {config['early_stopping_patience']} patience")
        
        # 학습 실행
        trainer.train()
        
        # 📈 최종 평가
        eval_results = trainer.evaluate()
        print(f"✅ {config['name']} 최종 성능 (Listwise):")
        print(f"   Accuracy: {eval_results['eval_accuracy']:.4f}")
        
        # 모델 저장
        model_save_path = f"./results_{config['name']}_listwise/final"
        trainer.save_model(model_save_path)
        
        print(f"💾 {config['name']} 모델 저장 완료!")
        
        return {
            'model': model,
            'trainer': trainer,
            'config': config,
            'save_path': model_save_path,
            'final_accuracy': eval_results['eval_accuracy']
        }
    
    # 나머지 메서드들은 기존과 동일
    def generate_meta_features(self, models, data, device):
        """메타 특성 생성 (K-Fold 교차 검증)"""
        print("\n🧠 메타 특성 생성 중...")
        
        # 데이터를 DataFrame으로 변환
        df = pd.DataFrame(data)
        
        # K-Fold 설정
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)
        
        # 메타 특성 저장할 배열
        meta_features = np.zeros((len(data), len(models) * 24))
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
            print(f"  Fold {fold + 1}/{self.n_folds} 처리 중...")
            
            val_data = df.iloc[val_idx]
            
            for model_idx, model_info in enumerate(models):
                model = model_info['model']
                model.eval()
                
                tokenizer = AutoTokenizer.from_pretrained(
                    model_info['config']['model_name'],
                    cache_dir='C:/huggingface_cache'
                )
                
                fold_predictions = []
                
                for _, row in val_data.iterrows():
                    text = row['text']
                    
                    inputs = tokenizer(
                        text,
                        return_tensors="pt",
                        truncation=True,
                        padding=True,
                        max_length=512
                    ).to(device)
                    
                    with torch.no_grad():
                        outputs = model(**inputs)
                        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                        fold_predictions.append(probabilities.cpu().numpy()[0])
                
                start_col = model_idx * 24
                end_col = (model_idx + 1) * 24
                meta_features[val_idx, start_col:end_col] = np.array(fold_predictions)
        
        return meta_features
    
    def train_meta_model(self, meta_features, labels):
        """메타 모델 학습"""
        print("\n🎯 메타 모델 학습 중...")
        
        self.meta_model = lgb.LGBMClassifier(
            objective='multiclass',
            num_class=24,
            boosting_type='gbdt',
            num_leaves=31,
            learning_rate=0.05,
            feature_fraction=0.9,
            bagging_fraction=0.8,
            bagging_freq=5,
            verbose=0,
            random_state=self.random_state
        )
        
        self.meta_model.fit(meta_features, labels)
        print("✅ 메타 모델 학습 완료!")
        
        return self.meta_model
    
    def fit(self, train_data, valid_data, device):
        """전체 스택킹 앙상블 학습"""
        print("🚀 Listwise Loss 기반 Stacking 앙상블 학습 시작!")
        
        train_df = pd.DataFrame(train_data)
        valid_df = pd.DataFrame(valid_data)
        
        train_dataset = Dataset.from_pandas(train_df)
        valid_dataset = Dataset.from_pandas(valid_df)
        
        # 1단계: Listwise Loss 베이스 모델들 학습
        print("\n📚 1단계: Listwise Loss 베이스 모델들 학습")
        for config in self.base_model_configs:
            tokenizer = AutoTokenizer.from_pretrained(
                config['model_name'],
                cache_dir='C:/huggingface_cache'
            )
            print(f"✅ {config['name']} 토크나이저 로드 완료")
            
            def tokenize_function(examples):
                return tokenizer(
                    examples["text"],
                    truncation=True,
                    padding=True,
                    max_length=512
                )
            
            current_train_dataset = train_dataset.map(tokenize_function, batched=True)
            current_valid_dataset = valid_dataset.map(tokenize_function, batched=True)
            
            current_train_dataset = current_train_dataset.remove_columns(["text", "original_sentences", "answer"])
            current_valid_dataset = current_valid_dataset.remove_columns(["text", "original_sentences", "answer"])
            
            model_info = self.train_base_model(config, current_train_dataset, current_valid_dataset, tokenizer, device)
            self.base_models.append(model_info)
        
        # 📊 베이스 모델 성능 요약
        print("\n📈 Listwise Loss 베이스 모델 성능 요약:")
        total_accuracy = 0
        for model_info in self.base_models:
            accuracy = model_info['final_accuracy']
            total_accuracy += accuracy
            print(f"  {model_info['config']['name']}: {accuracy:.4f}")
        avg_accuracy = total_accuracy / len(self.base_models)
        print(f"  평균 정확도: {avg_accuracy:.4f}")
        
        # 2단계: 메타 특성 생성
        print("\n🔧 2단계: 메타 특성 생성")
        meta_features = self.generate_meta_features(self.base_models, train_data, device)
        labels = [item['label'] for item in train_data]
        
        # 3단계: 메타 모델 학습
        print("\n🎓 3단계: 메타 모델 학습")
        self.train_meta_model(meta_features, labels)
        
        print("\n🎉 Listwise Loss 기반 Stacking 앙상블 학습 완료!")
        print(f"🏆 예상 앙상블 성능: {avg_accuracy + 0.08:.4f} (Listwise 효과 + 메타 모델)")
        
        return self
    
    def predict(self, test_data, device):
        """스택킹 앙상블 예측"""
        print("\n🔮 Listwise Loss 기반 앙상블 예측 중...")
        
        test_meta_features = []
        
        for model_info in self.base_models:
            model = model_info['model']
            model.eval()
            
            tokenizer = AutoTokenizer.from_pretrained(
                model_info['config']['model_name'],
                cache_dir='C:/huggingface_cache'
            )
            
            model_predictions = []
            model_name = model_info['config']['name']
            
            for text in tqdm(test_data, desc=f"{model_name} 예측"):
                inputs = tokenizer(
                    text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512
                ).to(device)
                
                with torch.no_grad():
                    outputs = model(**inputs)
                    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                    model_predictions.append(probabilities.cpu().numpy()[0])
            
            test_meta_features.append(np.array(model_predictions))
        
        final_meta_features = np.hstack(test_meta_features)
        final_predictions = self.meta_model.predict(final_meta_features)
        final_probabilities = self.meta_model.predict_proba(final_meta_features)
        
        print("✅ Listwise Loss 기반 예측 완료!")
        
        return final_predictions, final_probabilities

In [7]:
print("디바이스 설정 중...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

# GPU 메모리 사용량 체크 함수
def print_gpu_utilization():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        usage_percent = (allocated / total) * 100
        print(f"GPU 메모리: {allocated:.2f}GB 사용 / {total:.2f}GB 전체 ({usage_percent:.1f}% 사용)")
    else:
        print("CUDA 사용 불가")

print_gpu_utilization()

디바이스 설정 중...
디바이스: cuda
GPU 메모리: 0.00GB 사용 / 8.00GB 전체 (0.0% 사용)


In [ ]:
# Stacking 앙상블 생성 및 학습
stacking_ensemble = AdvancedStackingEnsemble(n_folds=5, random_state=42)

# 학습 실행 (tokenizer 파라미터 제거)
stacking_ensemble.fit(train_data, valid_data, device)

🚀 Listwise Loss 기반 Stacking 앙상블 학습 시작!

📚 1단계: Listwise Loss 베이스 모델들 학습
✅ electra_small_listwise 토크나이저 로드 완료


Map: 100%|██████████| 5881/5881 [00:00<00:00, 6614.16 examples/s]



🔥 electra_small_listwise 모델 학습 시작 (Listwise Loss 적용)...


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🎓 electra_small_listwise Listwise 설정:
   Learning Rate: 3e-05
   Batch Size: 64
   Epochs: 45
   Loss Function: ListMLE (Listwise Ranking)
   Early Stopping: 8 patience


...

Map: 100%|██████████| 5881/5881 [00:00<00:00, 8712.27 examples/s]



🔥 bert_base_listwise 모델 학습 시작 (Listwise Loss 적용)...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🎓 bert_base_listwise Listwise 설정:
   Learning Rate: 2e-05
   Batch Size: 32
   Epochs: 40
   Loss Function: ListMLE (Listwise Ranking)
   Early Stopping: 8 patience


Step,Training Loss,Validation Loss,Accuracy
100,3.201000,3.179537,0.046188


In [ ]:
# =============================================================================
# 구조 인식 투표 예측기 클래스 (기존 코드에 추가)
# =============================================================================

class StructureAwareVotingPredictor:
    """문장 순서 배열을 위한 구조 인식 투표 예측기"""
    
    def __init__(self, stacking_ensemble, label_to_perm, perm_to_label):
        self.ensemble = stacking_ensemble
        self.label_to_perm = label_to_perm
        self.perm_to_label = perm_to_label
        
    def predict(self, test_texts, device):
        """구조 인식 투표 예측 실행"""
        print("🎯 구조 인식 투표 예측 시작...")
        
        # 각 베이스 모델별 예측 수집
        all_model_predictions = []
        all_model_probabilities = []
        
        for model_idx, model_info in enumerate(self.ensemble.base_models):
            print(f"  모델 {model_idx + 1}/{len(self.ensemble.base_models)} 예측 중...")
            
            model = model_info['model']
            model.eval()
            
            tokenizer = AutoTokenizer.from_pretrained(
                model_info['config']['model_name'],
                cache_dir='C:/huggingface_cache'
            )
            
            model_predictions = []
            model_probabilities = []
            
            for text in tqdm(test_texts, desc=f"모델 {model_idx + 1}"):
                inputs = tokenizer(
                    text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512
                ).to(device)
                
                with torch.no_grad():
                    outputs = model(**inputs)
                    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                    prob_array = probabilities.cpu().numpy()[0]
                    
                    model_predictions.append(np.argmax(prob_array))
                    model_probabilities.append(prob_array)
            
            all_model_predictions.append(model_predictions)
            all_model_probabilities.append(model_probabilities)
        
        # 구조 인식 투표 실행
        print("  🧠 문장 쌍별 관계 분석 및 최적 순열 구성 중...")
        final_predictions = []
        final_probabilities = []
        
        for i in tqdm(range(len(test_texts)), desc="구조 분석"):
            # 각 모델의 예측을 순열로 변환
            model_permutations = []
            model_confidences = []
            
            for model_idx in range(len(self.ensemble.base_models)):
                pred_label = all_model_predictions[model_idx][i]
                pred_perm = self.label_to_perm[pred_label]
                confidence = np.max(all_model_probabilities[model_idx][i])
                
                model_permutations.append(pred_perm)
                model_confidences.append(confidence)
            
            # 문장 쌍별 순서 관계 분석
            pair_votes = {}
            for pos1 in range(4):
                for pos2 in range(4):
                    if pos1 != pos2:
                        pair_votes[(pos1, pos2)] = []
            
            # 각 모델의 순열에서 쌍별 관계 추출
            for model_idx, perm in enumerate(model_permutations):
                confidence = model_confidences[model_idx]
                
                for pos1 in range(4):
                    for pos2 in range(4):
                        if pos1 != pos2:
                            # perm[pos1]이 perm[pos2]보다 먼저 나와야 하는지 확인
                            sent1_order = perm[pos1]
                            sent2_order = perm[pos2]
                            
                            if sent1_order < sent2_order:
                                # 문장 pos1이 문장 pos2보다 먼저
                                pair_votes[(pos1, pos2)].append(confidence)
                            else:
                                # 문장 pos2가 문장 pos1보다 먼저
                                pair_votes[(pos2, pos1)].append(confidence)
            
            # 쌍별 투표 결과로 최적 순열 구성
            best_permutation = self._construct_optimal_permutation(pair_votes)
            
            # 최적 순열을 라벨로 변환
            if best_permutation in self.perm_to_label:
                final_label = self.perm_to_label[best_permutation]
            else:
                # 백업: 가장 높은 신뢰도의 모델 선택
                best_model_idx = np.argmax(model_confidences)
                final_label = all_model_predictions[best_model_idx][i]
            
            final_predictions.append(final_label)
            
            # 확률은 해당 순열에 대한 모든 모델의 평균 확률
            avg_probs = np.mean([all_model_probabilities[j][i] for j in range(len(self.ensemble.base_models))], axis=0)
            final_probabilities.append(avg_probs)
        
        final_predictions = np.array(final_predictions)
        final_probabilities = np.array(final_probabilities)
        
        print("✅ 구조 인식 투표 예측 완료!")
        return final_predictions, final_probabilities
    
    def _construct_optimal_permutation(self, pair_votes):
        """쌍별 투표 결과로부터 최적 순열 구성"""
        # 각 쌍에 대한 투표 강도 계산
        pair_strengths = {}
        for pair, votes in pair_votes.items():
            if votes:
                pair_strengths[pair] = sum(votes) / len(votes)
            else:
                pair_strengths[pair] = 0
        
        # 토너먼트 방식으로 순서 결정
        sentences = list(range(4))
        sentence_scores = {i: 0 for i in range(4)}
        
        for i in range(4):
            for j in range(4):
                if i != j:
                    if pair_strengths.get((i, j), 0) > pair_strengths.get((j, i), 0):
                        sentence_scores[i] += pair_strengths.get((i, j), 0)
                    else:
                        sentence_scores[j] += pair_strengths.get((j, i), 0)
        
        # 점수 순으로 정렬하여 순열 생성
        sorted_sentences = sorted(sentences, key=lambda x: sentence_scores[x], reverse=True)
        
        # 순열을 올바른 형태로 변환 (어떤 문장이 몇 번째 위치에 있는지)
        permutation = [0] * 4
        for position, sentence_idx in enumerate(sorted_sentences):
            permutation[sentence_idx] = position
        
        return tuple(permutation)

In [ ]:
# =============================================================================
# 수정된 예측 셀 코드 (기존 셀 8번 대체)
# =============================================================================

print("테스트 데이터 예측 중...")

# 테스트 데이터 로드
test_df = pd.read_csv("./test.csv")
print(f"테스트 데이터: {len(test_df)}개")

# 테스트 데이터 전처리
test_texts = []
for _, row in test_df.iterrows():
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    text = " [SEP] ".join(sentences)
    test_texts.append(text)

# 구조 인식 투표 예측기 생성
structure_predictor = StructureAwareVotingPredictor(
    stacking_ensemble, label_to_perm, perm_to_label
)

# 구조 인식 투표로 예측 실행
final_predictions, final_probabilities = structure_predictor.predict(test_texts, device)

# 예측 결과를 순열로 변환
predicted_orders = []
confidences = []

for i, pred_label in enumerate(final_predictions):
    predicted_order = list(label_to_perm[pred_label])
    confidence = np.max(final_probabilities[i])
    
    predicted_orders.append(predicted_order)
    confidences.append(confidence)

# 평균 신뢰도 출력
avg_confidence = np.mean(confidences)
print(f"\n평균 예측 신뢰도: {avg_confidence:.4f}")

# 구조 인식 투표의 효과 분석
print(f"\n🎯 구조 인식 투표 분석:")
print(f"   - 문장 쌍별 관계 기반 예측")
print(f"   - 베이스 모델 수: {len(stacking_ensemble.base_models)}개")
print(f"   - 신뢰도 가중 투표 적용")
print(f"   - 순열 일관성 검증 완료")

테스트 데이터 예측 중...
테스트 데이터: 1780개
🎯 구조 인식 투표 예측 시작...
  모델 1/3 예측 중...


모델 1: 100%|██████████| 1780/1780 [00:22<00:00, 79.88it/s]


  모델 2/3 예측 중...


모델 2: 100%|██████████| 1780/1780 [00:12<00:00, 144.06it/s]


  모델 3/3 예측 중...


모델 3: 100%|██████████| 1780/1780 [00:23<00:00, 74.33it/s]


  🧠 문장 쌍별 관계 분석 및 최적 순열 구성 중...


구조 분석: 100%|██████████| 1780/1780 [00:00<00:00, 12999.55it/s]

✅ 구조 인식 투표 예측 완료!

평균 예측 신뢰도: 0.6991

🎯 구조 인식 투표 분석:
   - 문장 쌍별 관계 기반 예측
   - 베이스 모델 수: 3개
   - 신뢰도 가중 투표 적용
   - 순열 일관성 검증 완료


In [ ]:
print("Submission 파일 생성 중...")

# 샘플 제출 파일 로드
sample_submission = pd.read_csv("./sample_submission.csv")

# 예측 결과를 제출 형식으로 변환
for i in range(4):
    sample_submission[f"answer_{i}"] = [pred[i] for pred in predicted_orders]

# 파일 저장
submission_filename = "stacking_ensemble_submission.csv"
sample_submission.to_csv(submission_filename, index=False)

print(f"제출 파일 저장 완료: {submission_filename}")

# 최종 결과 요약
print(f"\n" + "="*60)
print(f"🏆 고급 Stacking 앙상블 완료!")
print(f"📊 베이스 모델 수: {len(stacking_ensemble.base_models)}개")
print(f"🎯 메타 모델: LightGBM")
print(f"🔮 평균 예측 신뢰도: {avg_confidence*100:.2f}%")
print(f"📁 제출 파일: {submission_filename}")
print(f"="*60)

# 개별 모델 성능도 확인해보기
print(f"\n개별 베이스 모델 정보:")
for i, model_info in enumerate(stacking_ensemble.base_models):
    config = model_info['config']
    print(f"  모델 {i+1}: {config['name']} (LR: {config['learning_rate']}, Epochs: {config['epochs']})")

Submission 파일 생성 중...
제출 파일 저장 완료: stacking_ensemble_submission.csv

🏆 고급 Stacking 앙상블 완료!
📊 베이스 모델 수: 3개
🎯 메타 모델: LightGBM
🔮 평균 예측 신뢰도: 69.91%
📁 제출 파일: stacking_ensemble_submission.csv

개별 베이스 모델 정보:
  모델 1: bert_small (LR: 1.5e-05, Epochs: 25)
  모델 2: roberta_small (LR: 2e-05, Epochs: 20)
  모델 3: electra_small (LR: 3e-05, Epochs: 15)
